In [2]:

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())

In [1]:
import os
os.listdir('/home/hgidea/Desktop/Coding/Python/ai_ml practise/ml/internship/prodigy/archive/food-101/food-101/images')

In [2]:

import glob
import random
from PIL import Image

# Set the data directory path (assuming the folder is named "food-101" in your project directory)
data_dir = "/home/hgidea/Desktop/Coding/Python/ai_ml practise/ml/internship/prodigy/archive/food-101/food-101/images"

# Get all subdirectories (assuming these represent food classes)
class_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]

# Print total number of classes
print(f"Total number of classes: {len(class_dirs)}")

# Choose a random class directory
random_class_index = random.randint(0, len(class_dirs) - 1)
sample_class = class_dirs[random_class_index]

# Define the pattern to search for both .jpg and .png files
image_paths = glob.glob(os.path.join(data_dir, sample_class, "*.jpg")) + glob.glob(os.path.join(data_dir, sample_class, "*.png"))

# Print some information about the sample class
print(f"\nSample Class: {sample_class}")
print(f"Number of images: {len(image_paths)}")

# Optionally, display a few sample images (modify to show more if desired)
for i in range(min(3, len(image_paths))):  # Show the first 3 images or fewer if there are fewer images
    # Get the path of the current image
    image_path = image_paths[i]
    # Display the folder name and image name in terminal
    print(f"\nRandomly picked image from folder: {sample_class}, image name: {os.path.basename(image_path)}")
    try:
        # Open and display the image
        image = Image.open(image_path)
        image.show()  # This will display the image in a pop-up window
    except (IOError, OSError):
        print(f"Error displaying image {os.path.basename(image_path)}. Ensure PIL is installed and image format is supported.")

# Loop through each class directory
for class_dir in class_dirs:
    print(f"\nClass: {class_dir}")
    # Get the path to the current class directory
    class_path = os.path.join(data_dir, class_dir)
    # List all images for the current class directory
    image_files = [f for f in os.listdir(class_path) if f.endswith(".jpg") or f.endswith(".png")]
    # Print number of images for the current class
    print(f"Number of images: {len(image_files)}")
    # Print some example image filenames (optional)
    if len(image_files) > 0:
        print("Example images:")
        for i in range(min(3, len(image_files))):
            print(image_files[i])


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import psutil
import sys  # Import the sys module for sys.exit()

# Define memory threshold (e.g., 90% of available memory)
memory_threshold_percent = 98

def check_memory_usage():
    # Get current memory usage
    memory_usage = psutil.virtual_memory().percent
    return memory_usage

# Main program loop
while True:
    # Check memory usage before loading images
    memory_usage_before_loading = check_memory_usage()
    print("Memory Usage Before Loading:", memory_usage_before_loading)

    # Load and preprocess images
    images = []
    labels = []
    batch_size =  36 # You can adjust this based on your available memory

    # Loop through each class directory
    for class_index, class_dir in enumerate(class_dirs):
        # Get the path to the current class directory
        class_path = os.path.join(data_dir, class_dir)
        # List all images for the current class directory
        image_files = [f for f in os.listdir(class_path) if f.endswith(".jpg") or f.endswith(".png")]
        # Load and preprocess images in batches
        for i in range(0, len(image_files), batch_size):
            batch_image_files = image_files[i:i+batch_size]
            batch_images = []
            batch_labels = []
            for image_file in batch_image_files:
                # Construct the path to the current image
                image_path = os.path.join(class_path, image_file)
                # Load the image using OpenCV
                try:
                    image = cv2.imread(image_path)
                    # Resize the image to 224x224
                    image = cv2.resize(image, (224, 224))
                    # Convert the image to float32 and normalize pixel values to the range [0, 1]
                    image = image.astype(np.float32) / 255.0
                    # Append the image and its corresponding label to the batch lists
                    batch_images.append(image)
                    batch_labels.append(class_index)
                except Exception as e:
                    print(f"Error loading/preprocessing image: {image_path}, {e}")
            # Append the batch images and labels to the main lists
            images.extend(batch_images)
            labels.extend(batch_labels)

            # Check memory usage after processing each batch
            memory_usage_after_batch = check_memory_usage()
            print("Memory Usage After Processing Batch:", memory_usage_after_batch)
            # Check if memory usage exceeds the threshold
            if memory_usage_after_batch > memory_threshold_percent:
                print("Memory usage exceeded threshold. Terminating program.")
                # Perform any necessary cleanup or logging here
                # Terminate the program
                sys.exit()  # Use sys.exit() to terminate the program

        # Check memory usage after processing each class
        memory_usage_after_class = check_memory_usage()
        print("Memory Usage After Processing Class:", memory_usage_after_class)
        # Check if memory usage exceeds the threshold
        if memory_usage_after_class > memory_threshold_percent:
            print("Memory usage exceeded threshold. Terminating program.")
            # Perform any necessary cleanup or logging here
            # Terminate the program
            sys.exit()  # Use sys.exit() to terminate the program

    # Convert the lists to numpy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Split the dataset into training, validation, and testing sets
    # Here, we allocate 80% of the data for training, 10% for validation, and 10% for testing
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

    # Print the shapes of the training, validation, and testing sets
    print("Training set shape:", X_train.shape)
    print("Validation set shape:", X_val.shape)
    print("Testing set shape:", X_test.shape)

    # Program continues here if memory usage is below the threshold
    print("Program terminated.")
